## Data procesing
wiki + IMDB





In [0]:
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar

--2019-08-14 20:55:39--  https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 811315200 (774M) [application/x-tar]
Saving to: ‘wiki_crop.tar’

wiki_crop.tar       100%[===================>] 773.73M  26.3MB/s    in 30s     

2019-08-14 20:56:10 (25.6 MB/s) - ‘wiki_crop.tar’ saved [811315200/811315200]



In [0]:
!tar xf wiki_crop.tar
!rm wiki_crop.tar

In [0]:
import scipy.io
import os
import pandas as pd
import numpy as np
import cv2
from datetime import datetime

from numpy import asarray
from PIL import Image


mat = scipy.io.loadmat('./wiki_crop/wiki.mat')

instances = mat['wiki'][0][0][0].shape[1]
columns = ['dob', 'photo_taken', 'full_path', 'gender', "name", "face_location", "face_score", "second_face_score"]
df_wiki = pd.DataFrame(index=range(0, instances), columns=columns)
for i in mat:
  if i == "wiki":
    curr_array = mat[i][0][0]
    for j in range(len(curr_array)):
      df_wiki[columns[j]] = pd.DataFrame(curr_array[j][0])

In [0]:
#@title
mat = scipy.io.loadmat('./imdb_crop/imdb.mat')

instances_imdb = mat['imdb'][0][0][0].shape[1]
# columns = ['dob', 'photo_taken', 'full_path', 'gender', "name", "face_location", "face_score", "second_face_score"]
df_imdb = pd.DataFrame(index=range(0, instances_imdb), columns=columns)
for i in mat:
  if i == "imdb":
    curr_array = mat[i][0][0]
    print(len(curr_array))
    for j in range(0, len(curr_array)-2): #imdb has 10 columns
      df_imdb[columns[j]] = pd.DataFrame(curr_array[j][0])

In [0]:
 datetime.fromordinal(max(int(720044) - 366,1))

In [0]:
def cale_age(taken, dob):
  
  birth = datetime.fromordinal(max(int(dob) - 366,1))
  
  if birth.month < 7:
    return taken - birth.year
  else:
    return taken - birth.year - 1

df_wiki['age'] = [cale_age(df_wiki['photo_taken'][i], df_wiki['dob'][i]) for i in range(len(df_wiki['dob']))]
#df_imdb['age'] = [cale_age(df_imdb['photo_taken'][i], df_imdb['dob'][i]) for i in range(len(df_imdb['dob']))]

In [0]:
#clean data
#remove no face picture
df_wiki = df_wiki[df_wiki['face_score'] != -np.inf]
#remove more faces in one picuture
df_wiki = df_wiki[df_wiki['second_face_score'].isna()]
#threshold more than 3
df_wiki = df_wiki[df_wiki['face_score'] >= 3]
#remove no gender
df_wiki = df_wiki[df_wiki['gender'].isna()==False]
#reomve unuse columns
df_wiki = df_wiki.drop(columns=['name', 'face_score', 'second_face_score', 'face_location'])


In [0]:
#@title
#clean data
#remove no face picture
df_imdb = df_imdb[df_imdb['face_score'] != -np.inf]
#remove more faces in one picuture
df_imdb = df_imdb[df_imdb['second_face_score'].isna()]
#threshold more than 3
df_imdb = df_imdb[df_imdb['face_score'] >= 3]
#remove no gender
df_imdb = df_imdb[df_imdb['gender'].isna()==False]
#reomve unuse columns
df_imdb = df_imdb.drop(columns=['name', 'face_score', 'second_face_score', 'face_location'])


In [0]:
df_wiki.head()

In [0]:
df_wiki = df_wiki[df_wiki['age']>0]
df_wiki = df_wiki[df_wiki['age']<=100]

# df_imdb = df_imdb[df_imdb['age']>0]
# df_imdb = df_imdb[df_imdb['age']<=100]

In [0]:
target_size = (224,224)
#update to pixel value

def getImagetoPixels(image_path, db='wiki'): #db = wiki or imdb
  image = cv2.imread('{}_crop/{}'.format(db, image_path[0]), cv2.IMREAD_COLOR)
  image = cv2.resize(image,target_size)
#   print(db)
  return image.reshape(1,-1)[0]

In [0]:
# image = cv2.imread('./wiki_crop/00/10049200_1891-09-16_1958.jpg', cv2.IMREAD_COLOR)
# image = cv2.resize(image,target_size)
# image = getImagetoPixels(['00/10049200_1891-09-16_1958.jpg'])
# image.reshape(1,-1)[0]
df_wiki['pixels'] = df_wiki['full_path'].apply(getImagetoPixels)

In [0]:
#@title
df_imdb['pixels'] = df_imdb['full_path'].apply(getImagetoPixels)

In [0]:
df = df_wiki

In [0]:
del df_wiki
# del df_imdb

In [0]:
df.shape #(115549, 6)

(22138, 6)

In [0]:
# from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.layers import Layer, Input, Dropout, Conv2D, Activation, add, BatchNormalization, UpSampling2D, ZeroPadding2D, Conv2DTranspose, Flatten, MaxPooling2D, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.backend import mean
from keras.models import Model, Sequential
from keras.utils import plot_model, to_categorical
from keras.engine.topology import Network #for untrainable discrimator model but weight still is updated
from keras.callbacks import ModelCheckpoint, TensorBoard

Using TensorFlow backend.


In [0]:
target_classes = to_categorical(df['age'].values, 101)

In [0]:
image_features = []
for i in range(0, df.shape[0]):
  image_features.append(df['pixels'].values[i])

In [0]:
from tempfile import mkdtemp
import os.path as path

filename_array = path.join(mkdtemp(), 'image_features_array.dat')
fp_image_features_array = np.memmap(filename_array, dtype='float32', mode='w+',\
                                    shape=(len(image_features), 224*224*3))

In [0]:
# for i in range(0, len(image_features)):
fp_image_features_array[:] = np.array(image_features[:])
#   print(i)

In [0]:
del image_features

In [0]:
fp_image_features_array.flush()
# fp_image_features_array.close()

In [0]:
fp_image_features_array.shape

(22138, 150528)

In [0]:
from tempfile import mkdtemp
import os.path as path
filename = path.join(mkdtemp(), 'imageFeaturefile.dat')
fp = np.memmap(filename, dtype='float32', mode='w+', shape=(fp_image_features_array.shape[0],224,224,3))

In [0]:
fp[:] = fp_image_features_array.reshape(fp_image_features_array.shape[0], 224, 224, 3)

In [0]:
fp[:] = fp/255

In [0]:
train_x_mp = path.join(mkdtemp(), 'train_x.dat')
train_x = np.memmap(train_x_mp, dtype='float32', mode='w+', shape=(int(fp.shape[0] * 0.7),224,224,3))

train_y_mp = path.join(mkdtemp(), 'train_y.dat')
train_y = np.memmap(train_y_mp, dtype='float32', mode='w+', shape=(int(fp.shape[0] * 0.7),101))

# test_x_mp = path.join(mkdtemp(), 'test_x.dat')
# test_x = np.memmap(test_x_mp, dtype='float32', mode='w+', shape=(int(fp.shape[0] * 0.3),224,224,3))

# test_y_mp = path.join(mkdtemp(), 'test_y.dat')
# test_y = np.memmap(test_y_mp, dtype='float32', mode='w+', shape=(int(fp.shape[0] * 0.3),2))

test_x_mp = path.join(mkdtemp(), 'test_x.dat')
test_x = np.memmap(test_x_mp, dtype='float32', mode='w+', shape=(6642,224,224,3))

test_y_mp = path.join(mkdtemp(), 'test_y.dat')
test_y = np.memmap(test_y_mp, dtype='float32', mode='w+', shape=(6642,101))

In [0]:
from sklearn.model_selection import train_test_split
train_x[:], test_x[:], train_y[:], test_y[:] = train_test_split(fp, target_classes, test_size=0.3)

In [0]:
test_x.shape

(6642, 224, 224, 3)

## Involving VGG FACE
https://github.com/rcmalli/keras-vggface.git

In [0]:
pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-90xiuvzg
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-90xiuvzg
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=8d32bb627ef02e1952912ca07de4fafb2a4a08a01fa59f774c74c577dac9e131
  Stored in directory: /tmp/pip-ephem-wheel-cache-fph56x3h/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [0]:
from keras_vggface.vggface import VGGFace
vgg_model = VGGFace(include_top=False, model='vgg16', weights='vggface', input_shape=(224, 224, 3))

W0814 21:12:45.092542 140266772465536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0814 21:12:45.310530 140266772465536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 21:12:45.365767 140266772465536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0814 21:12:45.462373 140266772465536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



58916864/58909280 [==============================] - 1s 0us/step


W0814 21:12:47.400665 140266772465536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0814 21:12:47.402165 140266772465536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [0]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [0]:
#add top layers
last_layer  = vgg_model.get_layer('pool5').output
for layer in vgg_model.layers:
  layer.trainable = False

x = Conv2D(4096, (7, 7), activation='relu')(last_layer)
x = Dropout(0.5)(x)
x = Conv2D(4096, (1, 1), activation='relu')(x)
x = Dropout(0.5)(x)
x = Conv2D(101, (1,1),name="prediction")(x) #for age estimator

x = Flatten(name='flatten')(x)
output = Activation('softmax')(x)

facegender_vgg_model = Model(vgg_model.input, output)

W0814 21:12:51.078122 140266772465536 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
facegender_vgg_model.compile(loss='categorical_crossentropy', \
                          optimizer=Adam(), \
                          metrics=['accuracy'])

W0814 21:13:25.842913 140266772465536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
for layer in facegender_vgg_model.layers:
    print(layer, layer.trainable)

In [0]:
checkpointer = ModelCheckpoint(filepath='age_model.h5', \
                               monitor= 'val_loss', \
                               verbose=1, \
                               save_best_only=True,\
                               mode = 'auto'
                               )

In [0]:
loss_track = TensorBoard(log_dir='./face_age_estimator_logs', histogram_freq=0, \
                            batch_size=256, \
                            write_graph=True,\
                            write_grads=False, \
                            write_images=False, \
                            embeddings_freq=0, \
                            embeddings_layer_names=None, \
                            embeddings_metadata=None,\
                            embeddings_data=None,\
                            update_freq='epoch')

In [0]:
epochs = 250
batch_size = 256
scores = []
for i in range(epochs):

  ix_train = np.random.choice(train_x.shape[0], size=batch_size)
  score = faceage_vgg_model.fit(train_x[ix_train], train_y[ix_train]
 , epochs=1, validation_data=(test_x, test_y), callbacks=[checkpointer])
  
  print(i)
  
  histories.append(score)

W0814 21:13:41.430041 140266772465536 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 256 samples, validate on 6642 samples
Epoch 1/1
256/256 [==============================] - 93s 365ms/step - loss: 5.0217 - acc: 0.0195 - val_loss: 4.4094 - val_acc: 0.0372

Epoch 00001: val_loss improved from inf to 4.40944, saving model to age_model.h5
0
Train on 256 samples, validate on 6642 samples
Epoch 1/1
256/256 [==============================] - 82s 321ms/step - loss: 4.3478 - acc: 0.0430 - val_loss: 4.1948 - val_acc: 0.0367

Epoch 00001: val_loss improved from 4.40944 to 4.19482, saving model to age_model.h5
1
Train on 256 samples, validate on 6642 samples
Epoch 1/1
256/256 [==============================] - 83s 323ms/step - loss: 4.2216 - acc: 0.0352 - val_loss: 4.0776 - val_acc: 0.0370

Epoch 00001: val_loss improved from 4.19482 to 4.07765, saving model to age_model.h5
2
Train on 256 samples, validate on 6642 samples
Epoch 1/1
256/256 [==============================] - 83s 323ms/step - loss: 3.9617 - acc: 0.0234 - val_loss: 3.9899 - val_acc: 0.0382

Epoch 00001: va

In [0]:
import time

time.ctime()
xian
#10:30 ~ 15:14

'Thu Aug 15 03:01:59 2019'